# **Tropos (API Feedback Generation)**
#### *This notebook handles set up to Github/Google Drive, model feedback generation, and generates docx files with feedback.*

## **--Set up**

In [ ]:
#######################################################################
# Sets up Google Drive, Github, Github branch
# Installs requirements.txt and needed libraries
#######################################################################
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b fix/RateLimitError origin/fix/RateLimitError

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet #for chatgpt
!pip install -q google-generativeai --quiet #for gemini
!pip install anthropic httpx --quiet #for claude

In [ ]:
########################################################################
# Default imports and connecting all API keys
#######################################################################
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
import anthropic
openai_key = userdata.get("conner").strip()
deepseek_key = userdata.get("deepseek").strip()
llama_key = userdata.get("groq").strip()
claude_key = userdata.get("claude").strip()


os.environ["OPENAI_API_KEY"] = openai_key

from openai import OpenAI
client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

os.environ["LLAMA_API_URL"] = "https://api.groq.com/openai/v1"  # For Ollama
os.environ["LLAMA_API_KEY"] = llama_key  #
os.environ["CLAUDE_API_KEY"] = claude_key

import google.generativeai as genai
genai.configure(api_key=userdata.get("gemini").strip())


## **--Feedback Generation**

In [ ]:
#######################################################################
# Ensure past generated output is cleared
# (comment out if do not want)
#######################################################################
from utils.file_utils import clear_directory_if_exists
output_dir = "/content/project/data/generated_output"
clear_directory_if_exists(output_dir)

In [14]:
#######################################################################
# Set up paths for feedback engine
#######################################################################
from tropos import test_feedback_console
reqs = "./data/raw/Requirements.docx"
examples = "./data/raw/Student_Submissions"
targets = "./data/unmarked_raw"
outputs = "./data/generated_output"

In [ ]:
#chatGPT
test_feedback_console("FewShot", "gpt-4o", reqs, examples, targets, outputs,

In [ ]:
from tropos import test_feedback_console
#chatGPT 4.1
test_feedback_console(
    prompt_type="FewShot",
    model="gpt-4.1",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    verbose=True, #move to 'False' if you dont want all the outpu
    max_examples=3 # Set the number of examples taken by the thingyy at once
)

In [ ]:
# from tropos import test_feedback_console
# #deepseek
# test_feedback_console(
#     prompt_type="FewShot",
#     model="deepseek-chat",
#     requirements_path="./data/raw/Requirements.docx",
#     example_dir="./data/raw/Student_Submissions",
#     target_dir="./data/unmarked_raw",
#     output_dir="./data/generated_output",
#     verbose=True, #move to 'False' if you dont want all the output to be visble
#     max_examples=3 # Set the number of examples taken by the thingyy at once
# )

In [ ]:
# from tropos import test_feedback_console
# #gemini
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gemini-1.5-pro-latest",
#     requirements_path="./data/raw/Requirements.docx",
#     example_dir="./data/raw/Student_Submissions",
#     target_dir="./data/unmarked_raw",
#     output_dir="./data/generated_output",
#     verbose=True, #move to 'False' if you dont want all the output to be visble
#     max_examples=3 # Set the number of examples taken by the thingyy at once
# )

In [ ]:
# from tropos import test_feedback_console
# #LLaMA using groq, free version so smaller rate limits
# test_feedback_console(
#     prompt_type="FewShot-Llama", #made a different prompt to get llama feedback correctly put into docx
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     requirements_path="./data/raw/Requirements.docx",
#     example_dir="./data/raw/Student_Submissions",
#     target_dir="./data/unmarked_raw",
#     output_dir="./data/generated_output",
#     verbose=True, #makes it so the output is printed to console too for testing purposes
#     max_examples=3 # Set the number of examples taken by the thingyy at once
# )

In [ ]:
# from tropos import test_feedback_console
# # Claude
# test_feedback_console(
#     prompt_type="FewShot",
#     model="claude-3-opus-20240229",
#     requirements_path="./data/raw/Requirements.docx",
#     example_dir="./data/raw/Student_Submissions",
#     target_dir="./data/unmarked_raw",
#     output_dir="./data/generated_output",
#     verbose=True, #makes it so the output is printed to console too for testing purposes
#     max_examples=3 # Set the number of examples taken by the thingyy at once
# )

**--Import modules (you’re working on)**
<br>*each of our classes will be what will merge to this notebook (I'm pretty sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

**--Lauren's bit that can run the feedback generator here to test**
- ended up modularizing code to prepare for gradio/UI use.
  - had all files include ability to use uploaded files, used local testing files as default to be able to test
  - created feedback_engine file to be able to create runnable program in notebook to test
  - TODO: create runnable feedback program for Gradio/UI when that time comes